In [ ]:
from Preprocessamento import preprocessamento_dataframe
from segmentandoDatasets import *
from Modelos import *

In [ ]:
segmentacao_PKLot(imagens_treino=1024, dias_treino=2, imagens_validacao=64, dias_validaco=1, imagens_teste=None, dias_teste=None, faculdades=["PUC"])
val, _ = preprocessamento_dataframe(caminho_csv='CSV/PUC/PUC_Segmentado_Validacao.csv', autoencoder=False, data_algumentantation=False)
teste_puc, teste_df_puc = preprocessamento_dataframe(caminho_csv='CSV/PUC/PUC_Segmentado_Teste.csv', autoencoder=False, data_algumentantation=False)

In [ ]:
resultado = treina_modelos_em_batch('Modelo_Kyoto', 'Kyoto', 'CSV/PUC/PUC_Segmentado_Treino.csv', val, teste_puc, teste_df_puc, True, 10)

Contra UFPR04

In [ ]:
segmentacao_PKLot(imagens_treino=2, dias_treino=1, imagens_validacao=2, dias_validaco=1, imagens_teste=None, dias_teste=None, faculdades=["UFPR04"])
teste_UFPR04, teste_df_UFPR04 = preprocessamento_dataframe(caminho_csv='CSV/UFPR04/UFPR04_Segmentado_Teste.csv', autoencoder=False, data_algumentantation=False)

In [ ]:
testa_modelos('Modelo_Kyoto', teste_UFPR04, teste_df_UFPR04)

Contra UFPR05

In [ ]:
segmentacao_PKLot(imagens_treino=500, dias_treino=2, imagens_validacao=64, dias_validaco=1, imagens_teste=None, dias_teste=None, faculdades=["UFPR05"])
teste_UFPR05, teste_df_UFPR05 = preprocessamento_dataframe(caminho_csv='CSV/UFPR05/UFPR05_Segmentado_Teste.csv', autoencoder=False, data_algumentantation=False)

In [ ]:
testa_modelos('Modelo_Kyoto', teste_UFPR05, teste_df_UFPR05)